In [1]:
import cv2
import os
import numpy as np
# from google.colab import files
import matplotlib.pyplot as plt
from scipy.optimize import least_squares

print(f"Usando OpenCV versão: {cv2.__version__}")

# from google.colab.patches import cv2_imshow

Usando OpenCV versão: 4.12.0


In [ ]:
# !sudo apt update
# !sudo apt install -y colmap

In [ ]:
os.makedirs("images", exist_ok=True)
uploaded = files.upload()  # selecione múltiplas imagens
for fn in uploaded.keys():
    os.rename(fn, f"images/{fn}")

## Código adaptativo

In [106]:
import subprocess
import os

CENARIO = "boneco-compressed"  # boneco-compressed, boneco2, boneco3-compressed, cenario3-dinofauro, cenario4-vaso, cenario5-bulbassauro, cenario5-bulbassauro2
BASE_PATH = "./seg_fotos/"
# BASE_PATH = "./fotos/"
IMAGES_PATH = BASE_PATH + CENARIO + "/"
DATABASE_PATH = f"./processed/{CENARIO}/database.db"
OUTPUT_PATH = f"./processed/{CENARIO}"
os.makedirs(OUTPUT_PATH, exist_ok=True)

In [107]:
env_clean = os.environ.copy()
if 'LD_LIBRARY_PATH' in env_clean:
    del env_clean['LD_LIBRARY_PATH']

In [108]:
#feature extraction
comando = f"colmap feature_extractor --database_path {DATABASE_PATH} --image_path {IMAGES_PATH} --ImageReader.single_camera 1 \
    --SiftExtraction.max_image_size 3400 "
subprocess.run(comando, shell=True, env=env_clean)

I0927 23:19:53.617074 323216 misc.cc:44] 
Feature extraction
I0927 23:19:53.618438 323233 sift.cc:720] Creating SIFT GPU feature extractor
I0927 23:19:53.619992 323234 feature_extraction.cc:260] Processed file [1/124]
I0927 23:19:53.620007 323234 feature_extraction.cc:263]   Name:            IMG_20250924_105705091.jpg
E0927 23:19:53.620010 323234 feature_extraction.cc:267] IMG_20250924_105705091.jpg IMAGE_EXISTS: Features for image were already extracted.
I0927 23:19:53.620060 323234 feature_extraction.cc:260] Processed file [2/124]
I0927 23:19:53.620065 323234 feature_extraction.cc:263]   Name:            IMG_20250924_105708449.jpg
E0927 23:19:53.620067 323234 feature_extraction.cc:267] IMG_20250924_105708449.jpg IMAGE_EXISTS: Features for image were already extracted.
I0927 23:19:53.620126 323234 feature_extraction.cc:260] Processed file [3/124]
I0927 23:19:53.620129 323234 feature_extraction.cc:263]   Name:            IMG_20250924_105711284.jpg
E0927 23:19:53.620131 323234 feature_e

CompletedProcess(args='colmap feature_extractor --database_path ./processed/boneco-compressed/database.db --image_path ./seg_fotos/boneco-compressed/ --ImageReader.single_camera 1     --SiftExtraction.max_image_size 3400 ', returncode=0)

In [109]:
# Emparelhamento de características
comando = f"colmap exhaustive_matcher --database_path {DATABASE_PATH}"
subprocess.run(comando, shell=True, env=env_clean)

I0927 23:19:55.223232 323258 misc.cc:44] 
Feature matching & geometric verification
I0927 23:19:55.320008 323259 feature_matching_utils.cc:78] Bind FeatureMatcherWorker to GPU device 0
I0927 23:19:55.320363 323259 sift.cc:1446] Creating SIFT GPU feature matcher
I0927 23:19:55.321756 323258 pairing.cc:180] Generating exhaustive image pairs...
I0927 23:19:55.321765 323258 pairing.cc:213] Processing block [1/3, 1/3]
I0927 23:19:55.328977 323258 feature_matching.cc:117] in 0.007s
I0927 23:19:55.328991 323258 pairing.cc:213] Processing block [1/3, 2/3]
I0927 23:19:55.335474 323258 feature_matching.cc:117] in 0.006s
I0927 23:19:55.335487 323258 pairing.cc:213] Processing block [1/3, 3/3]
I0927 23:19:55.337041 323258 feature_matching.cc:117] in 0.002s
I0927 23:19:55.337055 323258 pairing.cc:213] Processing block [2/3, 1/3]
I0927 23:19:55.343040 323258 feature_matching.cc:117] in 0.006s
I0927 23:19:55.343051 323258 pairing.cc:213] Processing block [2/3, 2/3]
I0927 23:19:55.348793 323258 featur

CompletedProcess(args='colmap exhaustive_matcher --database_path ./processed/boneco-compressed/database.db', returncode=0)

In [110]:
# Mapper
comando = f"mkdir -p {OUTPUT_PATH}/sparse"
subprocess.run(comando, shell=True, env=env_clean)

comando = f"colmap mapper --database_path {DATABASE_PATH} --image_path {IMAGES_PATH} --output_path {OUTPUT_PATH}/sparse/ --Mapper.num_threads 8"
subprocess.run(comando, shell=True, env=env_clean)

I0927 23:19:57.692242 323292 incremental_pipeline.cc:260] Loading database
I0927 23:19:57.693367 323292 database_cache.cc:67] Loading rigs...
I0927 23:19:57.693400 323292 database_cache.cc:77]  1 in 0.000s
I0927 23:19:57.693405 323292 database_cache.cc:85] Loading cameras...
I0927 23:19:57.693414 323292 database_cache.cc:103]  1 in 0.000s
I0927 23:19:57.693419 323292 database_cache.cc:111] Loading frames...
I0927 23:19:57.693557 323292 database_cache.cc:128]  124 in 0.000s
I0927 23:19:57.693562 323292 database_cache.cc:136] Loading matches...
I0927 23:19:57.697888 323292 database_cache.cc:141]  2542 in 0.004s
I0927 23:19:57.697896 323292 database_cache.cc:157] Loading images...
I0927 23:19:57.702525 323292 database_cache.cc:241]  124 in 0.005s (connected 124)
I0927 23:19:57.702540 323292 database_cache.cc:252] Building correspondence graph...
I0927 23:19:57.716399 323292 database_cache.cc:279]  in 0.014s (ignored 0)
I0927 23:19:57.716938 323292 timer.cc:90] Elapsed time: 0.000 [minutes

CompletedProcess(args='colmap mapper --database_path ./processed/boneco-compressed/database.db --image_path ./seg_fotos/boneco-compressed/ --output_path ./processed/boneco-compressed/sparse/ --Mapper.num_threads 8', returncode=0)

In [111]:
# Isso vai salvar os pontos em um txt na pasta sparse/0
comando = f"colmap model_converter --input_path {OUTPUT_PATH}/sparse/0 --output_path {OUTPUT_PATH}/sparse/0 --output_type TXT"
subprocess.run(comando, shell=True, env=env_clean)

if os.path.exists(f'{OUTPUT_PATH}/sparse/1'):
    comando = f"colmap model_converter --input_path {OUTPUT_PATH}/sparse/1 --output_path {OUTPUT_PATH}/sparse/1 --output_type TXT"
    subprocess.run(comando, shell=True, env=env_clean)

In [112]:
# O CÓDIGO DESSA CÉLULA NÃO RODA NO COLAB POIS CRIA UM GRÁFICO INTERATIVO, TEM QUE RODAR LOCALMENTE
# Baixe o arquivo sparse/0/points3D.txt e coloque em uma pasta, na mesma pasta, crie um documento .py e coloque o seguinte código

# pip install open3d (primeiro instalar o open3D)

import open3d as o3d
import numpy as np

points = []
with open(f"{OUTPUT_PATH}/sparse/0/points3D.txt") as f:
    for line in f:
        if line.startswith("#"):
            continue
        elems = line.strip().split()
        x, y, z = map(float, elems[1:4])
        points.append([x, y, z])

points_np = np.array(points)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points_np)
o3d.visualization.draw_geometries([pcd])


You are using a GLFW raw input patch. This is not the official GLFW library.


In [113]:
# preprocessing (gera undistorted images)
# !colmap image_undistorter \
#     --image_path seg_fotos/cenario3-dinofauro/ \
#     --input_path processed/sparse/cenario3-dinofauro/0 \
#     --output_path dense/ \
#     --output_type COLMAP

comando = f"colmap image_undistorter --image_path {IMAGES_PATH} --input_path {OUTPUT_PATH}/sparse/0 \
    --output_path {OUTPUT_PATH}/dense/ --output_type COLMAP"
subprocess.run(comando, shell=True, env=env_clean)

I0927 23:20:33.164827 323630 misc.cc:44] 
Reading reconstruction
I0927 23:20:33.177050 323630 image.cc:364] => Reconstruction with 124 images and 9066 points
I0927 23:20:33.177075 323630 misc.cc:44] 
Image undistortion
I0927 23:20:33.178610 323630 undistortion.cc:197] Undistorting image [1/124]
I0927 23:20:33.307991 323630 undistortion.cc:197] Undistorting image [2/124]
I0927 23:20:33.311458 323630 undistortion.cc:197] Undistorting image [3/124]
I0927 23:20:33.311473 323630 undistortion.cc:197] Undistorting image [4/124]
I0927 23:20:33.311477 323630 undistortion.cc:197] Undistorting image [5/124]
I0927 23:20:33.338551 323630 undistortion.cc:197] Undistorting image [6/124]
I0927 23:20:33.338572 323630 undistortion.cc:197] Undistorting image [7/124]
I0927 23:20:33.338575 323630 undistortion.cc:197] Undistorting image [8/124]
I0927 23:20:33.338578 323630 undistortion.cc:197] Undistorting image [9/124]
I0927 23:20:33.338580 323630 undistortion.cc:197] Undistorting image [10/124]
I0927 23:2

CompletedProcess(args='colmap image_undistorter --image_path ./seg_fotos/boneco-compressed/ --input_path ./processed/boneco-compressed/sparse/0     --output_path ./processed/boneco-compressed/dense/ --output_type COLMAP', returncode=0)

In [114]:
# PatchMatch stereo
# !colmap patch_match_stereo \
#     --workspace_path dense \
#     --workspace_format COLMAP \
#     --PatchMatchStereo.geom_consistency true

comando = f"colmap patch_match_stereo --workspace_path {OUTPUT_PATH}/dense/ --workspace_format COLMAP --PatchMatchStereo.geom_consistency true"
subprocess.run(comando, shell=True, env=env_clean)

I0927 23:20:35.937744 323667 patch_match.cc:205] Reading workspace...
I0927 23:20:35.952296 323667 patch_match.cc:235] Reading configuration...
I0927 23:20:36.003871 323667 patch_match.cc:366] Configuration has 124 problems...
I0927 23:20:36.092741 323672 misc.cc:44] 
Processing view 1 / 124 for IMG_20250924_105705091.jpg
I0927 23:20:36.092784 323672 patch_match.cc:461] Reading inputs...
I0927 23:20:36.179085 323672 misc.cc:51] 
PatchMatch::Problem
-------------------
I0927 23:20:36.179095 323672 patch_match.cc:54] ref_image_idx: 0
I0927 23:20:36.179096 323672 patch_match.cc:55] src_image_idxs: 
I0927 23:20:36.179097 323672 patch_match.cc:58] 110 
I0927 23:20:36.179100 323672 patch_match.cc:58] 21 
I0927 23:20:36.179100 323672 patch_match.cc:58] 19 
I0927 23:20:36.179101 323672 patch_match.cc:58] 43 
I0927 23:20:36.179102 323672 patch_match.cc:58] 72 
I0927 23:20:36.179103 323672 patch_match.cc:58] 20 
I0927 23:20:36.179104 323672 patch_match.cc:58] 73 
I0927 23:20:36.179106 323672 pat

CompletedProcess(args='colmap patch_match_stereo --workspace_path ./processed/boneco-compressed/dense/ --workspace_format COLMAP --PatchMatchStereo.geom_consistency true', returncode=0)

In [115]:
# Fusion dos depth maps
# !colmap stereo_fusion \
#     --workspace_path dense/ \
#     --output_path dense/fused.ply

comando = f'colmap stereo_fusion --workspace_path {OUTPUT_PATH}/dense/ --output_path {OUTPUT_PATH}/dense/fused.ply'
subprocess.run(comando, shell=True, env=env_clean)

I0927 23:58:20.913332 337425 misc.cc:51] 
StereoFusion::Options
---------------------
I0927 23:58:20.913375 337425 fusion.cc:77] mask_path: 
I0927 23:58:20.913378 337425 fusion.cc:78] max_image_size: -1
I0927 23:58:20.913378 337425 fusion.cc:79] min_num_pixels: 5
I0927 23:58:20.913380 337425 fusion.cc:80] max_num_pixels: 10000
I0927 23:58:20.913381 337425 fusion.cc:81] max_traversal_depth: 100
I0927 23:58:20.913383 337425 fusion.cc:82] max_reproj_error: 2
I0927 23:58:20.913391 337425 fusion.cc:83] max_depth_error: 0.01
I0927 23:58:20.913393 337425 fusion.cc:84] max_normal_error: 10
I0927 23:58:20.913398 337425 fusion.cc:85] check_num_images: 50
I0927 23:58:20.913400 337425 fusion.cc:86] use_cache: 0
I0927 23:58:20.913401 337425 fusion.cc:87] cache_size: 32
I0927 23:58:20.913403 337425 fusion.cc:90] bbox_min: -3.40282e+38 -3.40282e+38 -3.40282e+38
I0927 23:58:20.913409 337425 fusion.cc:91] bbox_max: 3.40282e+38 3.40282e+38 3.40282e+38
I0927 23:58:20.913414 337425 fusion.cc:141] Reading 

CompletedProcess(args='colmap stereo_fusion --workspace_path ./processed/boneco-compressed/dense/ --output_path ./processed/boneco-compressed/dense/fused.ply', returncode=0)

In [116]:
# Código para visualizar o resultado do fused.ply
import open3d as o3d

# Carregar a nuvem de pontos
pcd = o3d.io.read_point_cloud(f"{OUTPUT_PATH}/dense/fused.ply")

# Visualizar a nuvem de pontos
o3d.visualization.draw_geometries([pcd])

You are using a GLFW raw input patch. This is not the official GLFW library.


In [123]:
# Surface reconstruction (opcional)
# !colmap poisson_mesher \
#     --input_path dense/fused.ply \
#     --output_path dense/meshed-poisson.ply \
#     --PoissonMeshing.trim 7 \
#     --PoissonMeshing.depth 13
trim  = 7
depth = 13

comando = f'colmap poisson_mesher --input_path {OUTPUT_PATH}/dense/fused.ply --output_path {OUTPUT_PATH}/dense/meshed-poisson.ply --PoissonMeshing.trim {trim} --PoissonMeshing.depth {depth}'
subprocess.run(comando, shell=True, env=env_clean)

Value Range: [1.749869,15.498216]


CompletedProcess(args='colmap poisson_mesher --input_path ./processed/boneco-compressed/dense/fused.ply --output_path ./processed/boneco-compressed/dense/meshed-poisson.ply --PoissonMeshing.trim 7 --PoissonMeshing.depth 13', returncode=0)

In [124]:
import open3d as o3d
mesh = o3d.io.read_triangle_mesh(f"{OUTPUT_PATH}/dense/meshed-poisson.ply")
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh])

You are using a GLFW raw input patch. This is not the official GLFW library.


COLMAP guarda tudo em um banco de dados SQLite

In [ ]:
!colmap feature_extractor \
   --database_path database.db \
   --image_path seg_fotos/fotos/cenario3-dinofauro/ \
   --ImageReader.single_camera 1 \
   --SiftExtraction.use_gpu 1 # GPU não funciona no colab

In [ ]:
#pra rodar local 

!colmap feature_extractor \
    --database_path database.db \
    --image_path seg_fotos/cenario3-dinofauro/ \
    --ImageReader.single_camera 1 \
    --SiftExtraction.max_image_size 3400 

Emparelhamento de características

In [ ]:
!colmap exhaustive_matcher \
--database_path database.db \
--SiftMatching.use_gpu 1

In [ ]:
# para rodar local
!colmap exhaustive_matcher \
--database_path database.db 

In [ ]:
# !mkdir processed/sparse/cenario3-dinofauro
!colmap mapper \
    --database_path database.db \
    --image_path fotos/cenario3-dinofauro/ \
    --output_path processed/sparse/cenario3-dinofauro

In [ ]:
# Isso vai salvar os pontos em um txt na pasta sparse/0

!colmap model_converter \
    --input_path processed/sparse/cenario3-dinofauro/0 \
    --output_path processed/sparse/cenario3-dinofauro/0 \
    --output_type TXT

In [22]:
# preprocessing (gera undistorted images)
!colmap image_undistorter \
    --image_path seg_fotos/cenario3-dinofauro/ \
    --input_path processed/sparse/cenario3-dinofauro/0 \
    --output_path dense/ \
    --output_type COLMAP

/bin/bash: /home/igor/anaconda3/envs/mc949/lib/libtinfo.so.6: no version information available (required by /bin/bash)
colmap: /home/igor/anaconda3/envs/mc949/lib/libcurl.so.4: no version information available (required by colmap)
colmap: symbol lookup error: /lib/libceres.so.2: undefined symbol: _ZN6google21kLogSiteUninitializedE


In [ ]:
# PatchMatch stereo
!colmap patch_match_stereo \
    --workspace_path dense \
    --workspace_format COLMAP \
    --PatchMatchStereo.geom_consistency true

In [ ]:
# Fusion dos depth maps
!colmap stereo_fusion \
    --workspace_path dense/ \
    --output_path dense/fused.ply
    # --workspace_format COLMAP \

In [ ]:
colmap stereo_fusion \
    --workspace_path dense \
    --output_path dense/fused.ply

In [ ]:
# Opcional: meshing
!colmap poisson_mesher \
    --input_path dense/fused.ply \
    --output_path dense/meshed-poisson.ply \
    --PoissonMeshing.trim 5 \
    --PoissonMeshing.depth 13

In [ ]:
import open3d as o3d

# Carregar nuvem de pontos
pcd = o3d.io.read_point_cloud("dense/fused.ply")

# Estimar normais (necessário para o Poisson)
pcd.estimate_normals()

# Gerar mesh Poisson
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=10)

# Salvar mesh
o3d.io.write_triangle_mesh("Trabalho 2/dense/meshed-poisson-o3d.ply", mesh)

# Visualizar
o3d.visualization.draw_geometries([mesh])

In [ ]:
# Isso é uma forma de visualizar de maneira fixa, a versão acima gera um gráfico iterativo

from mpl_toolkits.mplot3d import Axes3D

pts = []
with open("sparse/cenario3-dinofauro/0/points3D.txt") as f:
    for line in f:
        if line.startswith("#"): continue
        elems = line.strip().split()
        x, y, z = map(float, elems[1:4])
        pts.append([x, y, z])

pts_np = np.array(pts)
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection="3d")
ax.scatter(pts_np[:,0], pts_np[:,1], pts_np[:,2], s=1)
plt.show()

# Código antigo

## Detecção e Extração de Características

Usando SIFT

In [ ]:
"""
SIFT parameters
nfeatures:	Limits the max number of strongest keypoints returned. If too high, too many points; lower it to reduce keypoints.
contrastThreshold:	Higher values reduce keypoints in low-contrast areas. Default 0.04. Increase to detect fewer points.
edgeThreshold	Controls: edge response filtering. Higher values allow more edge points; lower removes edge-like keypoints.
nOctaveLayers:	More layers = finer scale-space detection; can increase number of keypoints slightly.
sigma:	Initial Gaussian blur. Larger sigma smooths image, reducing noise and small keypoints.
"""
sift = cv2.SIFT_create(
    nfeatures=5000,          # Maximum number of keypoints to retain
    nOctaveLayers=3,         # Number of layers in each octave (affects scale space granularity)
    contrastThreshold=0.04,  # Minimum contrast for keypoints
    edgeThreshold=10,        # Minimum edge response (reduces edge keypoints)
    sigma=1.2                # Gaussian smoothing applied at first octave
)

# Contains sift (keypoints, desc)
# sift = lista de tuplas:
# sift[0][0] = keypoint da imagem 0
# sift[0][1] = descriptor da imagem 0
# sift[1][0] = keypoint da imagem 1 (...)
sift = [sift.detectAndCompute(im, None) for im in images_cinza]

In [ ]:
"""
cv2.drawKeypoints(image, keypoints, outImage, color=None, flags=0) draws keypoints on an image.
Parameters:
image: Original BGR image.
keypoints: List of detected keypoints.
outImage: Output image. None means OpenCV creates a copy.
flags: How keypoints are drawn. Key options:
  cv2.DRAW_MATCHES_FLAGS_DEFAULT: Simple small circle.
  cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS: Draws size and orientation of keypoints (circles with lines).
  cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS: Ignores points with low response.
"""
img_sift = cv2.drawKeypoints(images[0], sift[0][0], None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
print(f"Number of keypoints found: {len(sift[0][0])}")
cv2_imshow(img_sift)


Usando ORB, que é **muito** mais rápido nesse caso

In [ ]:
"""
ORB parameters
nfeatures:	Maximum number of keypoints returned. Higher = more points.
scaleFactor:	Scale pyramid factor. Each level is prev_level / scaleFactor. Smaller factor → more levels → more keypoints.
nlevels:	Number of pyramid levels. More levels → detects features at more scales.
edgeThreshold:	Margin around image where keypoints are ignored. Prevents keypoints too close to edges.
firstLevel:	Base level of pyramid. Usually 0.
WTA_K:	Number of points used for computing BRIEF descriptor (2,3,4). Higher = more distinctive but slower.
scoreType:	How keypoints are ranked:
  cv2.ORB_HARRIS_SCORE → uses Harris corner score.
  cv2.ORB_FAST_SCORE → uses FAST score. |
  | patchSize | Size of patch around keypoint for descriptor computation. Larger = more context, slower. |
  | fastThreshold | Threshold for FAST corner detection. Higher → fewer keypoints; lower → more keypoints. |
"""
orb = cv2.ORB_create(
    nfeatures=5000,         # Max number of keypoints to retain
    scaleFactor=1.2,       # Pyramid decimation ratio between levels
    nlevels=4,             # Number of pyramid levels
    edgeThreshold=31,      # Size of border to ignore edges
    firstLevel=0,          # Index of first pyramid level
    WTA_K=2,               # Number of points for BRIEF descriptor (2,3,4)
    scoreType=cv2.ORB_HARRIS_SCORE,  # Method to rank keypoints
    patchSize=31,          # Size of patch used by BRIEF descriptor
    fastThreshold=20       # Threshold for FAST keypoint detection
)

# Contains orb (keypoints, desc)
orb = [orb.detectAndCompute(im, None) for im in images_cinza]

In [ ]:
img_orb = cv2.drawKeypoints(images[0], orb[0][0], None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
print(f"Number of keypoints found: {len(orb[0][0])}")
cv2_imshow(img_orb)

## Emparelhamento de Características

Brute-Force Matcher

In [ ]:
"""
Brute-Force Matcher

Brute-Force matcher is simple. It takes the descriptor of one feature in first set and is matched with all other features in second set using some distance calculation.
And the closest one is returned.

Lets take as an example images 1 and 2

"""

# BFMatcher with default params
bf = cv2.BFMatcher()
# Corrected indices to access the first two images (index 0 and 1)
matches = bf.knnMatch(sift[0][1], sift[1][1], k=2)

# Apply ratio test
good = []
for m,n in matches:
    if m.distance < 0.75 * n.distance:
        good.append([m])

# cv2.drawMatchesKnn expects list of lists as matches.
# Corrected indices for images and sift to match the first two images
matched_image = cv2.drawMatchesKnn(images[0], sift[0][0], images[1], sift[1][0], good, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

plt.imshow(matched_image),plt.show()

FLANN matcher

In [ ]:
"""
FLANN = Fast Library for Approximate Nearest Neighbors, optimized for large datasets and high-dimensional descriptors.

For FLANN based matcher, we need to pass two dictionaries which specifies the algorithm to be used, its related parameters etc.

Parameters:
  algorithm=1 → KD-tree (best for SIFT/SURF descriptors).
  trees=5 → number of KD-trees in the index (more trees = higher accuracy, slower).
  checks=50 → number of times to check trees for nearest neighbors (higher = more accurate, but takes more time).
"""
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)

flann = cv2.FlannBasedMatcher(index_params, search_params)

"""
knnMatch() finds the two nearest neighbors for each descriptor.
We use k=2 because Lowe’s ratio test requires two closest matches.

A descriptor is a numerical vector that describes the local image patch around a keypoint.
For SIFT, each descriptor is 128-dimensional.
Descriptors encode scale, orientation, and gradient information, so that we can compare keypoints across images.

matches = flann.knnMatch(des1, des2, k=2)
des1 → descriptors from image 1
des2 → descriptors from image 2
flann.knnMatch() compares all vectors in des1 to all vectors in des2 and finds the closest matches using Euclidean distance.
"""
matches = []
for i in range(len(sift) - 1):
  mtchs = flann.knnMatch(sift[i][1], sift[i + 1][1], k = 2) # Total matches
  # Lowe's tet ration
  """
  m = best match, n = second-best match.
  m.distance = Euclidean distance between descriptors.
  Ratio test keeps matches where best match is significantly closer than second-best.
  Threshold 0.7 is typical (adjustable).
  """
  good_matches = []
  for m, n in mtchs:
      if m.distance < 0.6 * n.distance:
          good_matches.append(m)

  print(f"{i} and {i+1} Good matches after ratio test: {len(good_matches)}")
  matches.append(good_matches)

In [ ]:
img_matches = cv2.drawMatches(
    images[2], sift[2][0],
    images[3], sift[3][0],
    matches[2], None,
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
)
"""
cv2.drawMatches() draws lines connecting matching keypoints.
flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS → only draws matched keypoints, not all keypoints.
"""
cv2_imshow(img_matches)